In [1]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/home/Jiahui/mob2crime'

In [8]:
import geopandas as gp
from shapely.geometry import Polygon
from src.utils import gis
from src.utils.gis import crs_normalization 

In [9]:
%matplotlib inline

In [10]:
polys1 = [
    Polygon([(0,0), (0,2), (2,2), (2,0)]),
    Polygon([(0,0), (0,-2), (-2,-2), (-2,0)]),
    
]
polys1=gp.GeoDataFrame(polys1, columns=['geometry'])

In [11]:
polys2 = [
    Polygon([(0,0), (0,1), (1,1), (1,0)]),
    Polygon([(1,0), (2,0), (2,1), (1,1)]),
    Polygon([(1,1), (1,2), (2,2), (2,1)]),
    Polygon([(0,0), (0,-1), (-1,-1), (-1,0)]),
    
]
polys2= gp.GeoDataFrame(polys2, columns=['geometry'])

In [12]:

def polys2polys(polys1, polys2, pname1='poly1', pname2='poly2', cur_crs=None, area_crs=None, intersection_only=True):
    """Compute the weights of from polygons 1 to polygons 2,
    So that the statistics in polys1 can be transferred to polys2

    If intersection_only:
        Weight(i,j) = Area(polys1i in polys2j) / Area(polys1i in polys2)
    Else:
        Weight(i,j) = Area(polys1i in polys2j) / Area(polys1i)

    :param polys1: GeoDataFrame
        polygons with statistics to distributed over the other polygons
    :param polys2: GeoDataFrame
        polygons to get statistics from polys1
    :param pname1: column name for the index of polys1 in the output
    :param pname2: column name for the index of polys2 in the output
    :param cur_crs: int, string, dict
        the current CRS of polys1 and polys2 (epsg code, proj4 string, or dictionary of projection parameters)
    :param area_crs: int, string, dict
        the equal-area CRS for the area computation

    :return: pd.DataFrame(columns=[pname1, pname2, 'weight'])
        the mapping from polys1 to polys2
    """

    do_crs_transform = True
    cur_crs = crs_normalization(cur_crs)
    area_crs = crs_normalization(area_crs)
    print (cur_crs, area_crs)
    # make sure CRS is set correctly
    if cur_crs is None and polys1.crs is None and polys2.crs is None:
        if area_crs is None:
            do_crs_transform = False
            print("No current epsg is specified. Area is computed directed in the current coordinates")
        else:
            raise ValueError('area epsg is specified, but the polygons have no CRS')

    if do_crs_transform:
        if area_crs is None:
            raise ValueError(
                "Need to do area transform, but area is not specified. "
                f"cur_crs is {cur_crs}, polys1.crs is {polys1.crs}, polys2.crs is {polys2.crs}"
            )
        if polys1.crs is None: polys1.crs = cur_crs
        if polys2.crs is None: polys2.crs = cur_crs
    
    print(polys1.crs)
    # get intersections between polys1 and polys2
    ps1tops2 = gp.sjoin(polys1, polys2)
    itxns = []
    for li, row in ps1tops2.iterrows():
        itxn = polys2.loc[row.index_right].geometry.intersection(polys1.loc[li].geometry)
        itxns.append({pname1: li, pname2: row.index_right, 'geometry': itxn})
    itxns = gp.GeoDataFrame(itxns)

    # get area of the intersections
    if do_crs_transform:
        itxns.crs = polys1.crs
        itxns_for_area = itxns.to_crs(area_crs)
    else:
        itxns_for_area = itxns
    itxns['iarea'] = itxns_for_area.geometry.apply(lambda x: x.area)
    itxns.drop(itxns[itxns['iarea'] == 0].index, inplace=True)

    # compute the weight
    if intersection_only:
        polys1_area = itxns.groupby(pname1).apply(lambda x: x['iarea'].sum()).to_frame()
    else:
        polys1_area = polys1.to_crs(area_crs).geometry.apply(lambda x: x.area).to_frame()
        polys1_area.index.name = pname1
    polys1_area = polys1_area
    polys1_area.columns = [pname1 + '_area']
    polys1_area.reset_index(inplace=True)
    itxns = itxns.merge(polys1_area)
    itxns['weight'] = itxns['iarea'] / itxns[pname1 + '_area']
    return gp.pd.DataFrame(itxns[[pname1, pname2, 'weight']])
      
i = polys2polys(polys1, polys2, cur_crs=4326, area_crs=6362, intersection_only=False)
i

{'init': 'epsg:4326'} {'init': 'epsg:6362'}
{'init': 'epsg:4326'}


,poly1,poly2,weight
0,1,3,0.248236
1,0,0,0.251699
2,0,1,0.251699
3,0,2,0.248313


In [13]:
i = gis.polys2polys(polys1, polys2, cur_crs=4326, area_crs=6362, intersection_only=False)
i

,poly1,poly2,weight
0,1,3,0.248236
1,0,0,0.251699
2,0,1,0.251699
3,0,2,0.248313
